In [2]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import nb_general as nb
import scipy.linalg as la
import non_backtracking_tools as nbt
import time
import sys
import sympy as sy

In [2]:
# Make graph
G = nx.Graph([[1,2],[2,3],[3,1],[3,4],[2,4]])
# make K
A = nx.adjacency_matrix(G).todense()
D = np.diag(list(dict(G.degree).values()))
K = np.block([[A,D-np.eye(D.shape[0])],[-np.eye(D.shape[0]),np.zeros_like(D)]])

In [3]:
mu_vals, mu_vecs = la.eig(K)
lambda_vals, lambda_vecs = la.eig(A)

In [4]:
mu_vecs[:,np.argsort(mu_vals)[-3]]

array([ -7.99024334e-16 +5.00000000e-01j,
        -6.08416932e-17 -8.66203294e-17j,
         1.91595179e-17 +4.91971467e-17j,
        -1.60589429e-19 -5.00000000e-01j,
        -5.00000000e-01 -8.73305282e-16j,
         2.32265345e-16 -7.86714394e-17j,
         4.93076885e-17 +3.93225472e-17j,   5.00000000e-01 +0.00000000e+00j])

In [5]:
# Calculate with formuula
formula = np.zeros(lambda_vals.shape[0]*2)
for i in range(0,len(formula),2):
    x = lambda_vecs[:,i//2]
    y2 = mu_vecs[mu_vecs.shape[0]//2:,0]
    y1 = mu_vecs[mu_vecs.shape[0]//2:,i+1]
    formula[i] = (lambda_vals[i//2]*(x.T@y1) + 
                  np.sqrt((lambda_vals[i//2]*(x.T@y1))**2
                          -4*x.T@(D-np.eye(D.shape[0]))@y1))/2
#     formula[i+1] = (lambda_vals[i//2]*(x.T@y2) - np.sqrt((lambda_vals[i//2]*(x.T@y2))**2-4*x.T@(D-np.eye(D.shape[0]))@y1))/2
print(formula)

[  8.98860183e-01   0.00000000e+00   1.21376870e-08   0.00000000e+00
   1.69842559e-08   0.00000000e+00   9.77411867e-01   0.00000000e+00]


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: ComplexWarning: Casting complex values to real discards the imaginary part
  if __name__ == '__main__':


In [6]:
# Get all possible y
y = np.zeros((lambda_vecs.shape[0],mu_vecs.shape[1]))
for i in range(mu_vecs.shape[1]):
    y[:,i] = mu_vecs[mu_vecs.shape[0]//2:,i]
print(y)

[[ -1.80869728e-01  -3.16227766e-01   9.10017989e-02   9.10017989e-02
    1.66745463e-01   1.66745463e-01  -1.44337567e-01  -1.44337567e-01
    1.47166768e-16   1.47166768e-16]
 [ -1.94807259e-01  -3.16227766e-01  -8.99787158e-02  -8.99787158e-02
   -2.11488680e-01  -2.11488680e-01   6.17669816e-16   6.17669816e-16
    1.02062073e-01   1.02062073e-01]
 [ -1.80869728e-01  -3.16227766e-01   9.10017989e-02   9.10017989e-02
    1.66745463e-01   1.66745463e-01   1.44337567e-01   1.44337567e-01
   -1.82844928e-16  -1.82844928e-16]
 [ -1.94807259e-01  -3.16227766e-01  -8.99787158e-02  -8.99787158e-02
   -2.11488680e-01  -2.11488680e-01   5.69296931e-16   5.69296931e-16
   -1.02062073e-01  -1.02062073e-01]
 [ -1.43527948e-01  -3.16227766e-01  -5.92387986e-02  -5.92387986e-02
    2.10821198e-01   2.10821198e-01  -1.12602857e-15  -1.12602857e-15
   -5.42611021e-17  -5.42611021e-17]]


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  after removing the cwd from sys.path.


In [7]:
possible = []
for i in range(y.shape[1]):
    x = lambda_vecs[:,0]
    possible.append((lambda_vals[0]*(x.T@y[:,i])+np.sqrt((lambda_vals[0]*(x.T@y[:,i]))**2-4*x.T@(D-np.eye(D.shape[0]))@y[:,i])/2))

In [8]:
np.real(lambda_vals)

array([ 3.32340428,  0.35792637, -1.        , -1.68133064, -1.        ])

In [9]:
np.real(possible)

array([ -1.50094411e-01,  -5.96176143e-01,   2.69174824e-01,
         2.69174824e-01,   5.22938640e-01,   5.22938640e-01,
        -4.61214993e-17,  -4.61214993e-17,   2.04042548e-08,
         2.04042548e-08])

In [10]:
A = np.array([[0,1,0,1],[1,0,1,1],[0,1,0,0],[1,1,0,0]])
la.eig(A)

(array([ 2.17008649+0.j,  0.31110782+0.j, -1.00000000+0.j, -1.48119430+0.j]),
 array([[  5.22720726e-01,   3.68160356e-01,   7.07106781e-01,
           3.02028137e-01],
        [  6.11628457e-01,  -2.53622791e-01,   5.95213736e-16,
          -7.49390492e-01],
        [  2.81845199e-01,  -8.15224745e-01,  -4.58144079e-16,
           5.05936655e-01],
        [  5.22720726e-01,   3.68160356e-01,  -7.07106781e-01,
           3.02028137e-01]]))

In [11]:
G = nx.Graph([[1,2],[2,3],[2,4],[3,4]])
A = nx.adjacency_matrix(G).todense()
D = np.diag(list(dict(G.degree).values()))
K = np.block([[A,D-np.eye(D.shape[0])],[-np.eye(D.shape[0]),np.zeros_like(D)]])

In [12]:
# Get eigenvalues
lambda_vals, lambda_vecs = la.eig(A)
mu_vals, mu_vecs = la.eig(K)

In [13]:
lambda_2 = lambda_vals[1]

In [14]:
mu_2 = mu_vals[3]

In [15]:
x = lambda_vecs[:,1]
y = mu_vecs[4:,3]

In [16]:
c = x.T@y
d = x.T@(D-np.eye(D.shape[0]))@y

In [17]:
(lambda_2*c+np.sqrt((lambda_2*c)**2-4*d*c))/(2*c)

(-0.68889215122611114-0j)

### Odd that it works with the negative

In [18]:
(lambda_2*c-np.sqrt((lambda_2*c)**2-4*d*c))/(2*c)

(0.99999996869209296-0j)

In [19]:
mu_2

(0.99999996869208796+0j)

In [20]:
mu_vals

array([  0.00000000e+00+0.j       ,  -5.00000000e-01+0.8660254j,
        -5.00000000e-01-0.8660254j,   9.99999969e-01+0.j       ,
         1.00000003e+00+0.j       ,  -5.00000000e-01+0.8660254j,
        -5.00000000e-01-0.8660254j,  -7.77193096e-17+0.j       ])

In [4]:
# Check formula multiple times
positive = []
negative = []
neither = []
avg_mu_bound = []
min_mu_bound = []
simple_upper = []
simple_lower = []
rho_bound = []

# Create progress bar
toolbar_width = 40

# setup toolbar
sys.stdout.write("[%s]" % (" " * toolbar_width))
sys.stdout.flush()
sys.stdout.write("\b" * (toolbar_width+1))

# Create 50 random graphs
for i in range(50):
    # Create random graph that is connected
    G = nx.random_partition_graph([np.random.randint(20,100),np.random.randint(20,100)],0.25,.05)    # Create adjacency matrix
    
    # Create adjacency matrix, D matrix, D^{-1} matrix
    A = nx.adjacency_matrix(G).todense()
    D = np.diag(list(dict(G.degree).values()))
    D_inv = np.diag([1/i for i in dict(G.degree).values()])
    
    # Create K
    K = np.block([[A,D-np.eye(D.shape[0])],[-np.eye(D.shape[0]),np.zeros_like(D)]])
    
    # Get eigenvalues and eigenvectors of adjacecny matrix and K matrix
    lambda_vals, lambda_vecs = la.eig(A)
    mu_vals, mu_vecs = la.eig(K)
    
    # Find the second largest eigenvalue of K
    idx = np.argsort(mu_vals)[-2]
    mu_2 = mu_vals[idx]
    
    # Find y
    y = mu_vecs[mu_vecs.shape[0]//2:,idx]
    
    # Find second largest eigenvalue of A
    idx = np.argsort(lambda_vals)[-2]
    lambda_2 = lambda_vals[idx]
    
    # Find x
    x = lambda_vecs[:,idx]
    
    # Calculate x^Ty and x^T(D-I)y
    c = x.T@y
    d = x.T@(D-np.eye(D.shape[0]))@y
    
    # Calculate average, minimum, and maximum degree
    d_bar = np.mean(list(dict(G.degree).values()))
    d_min = np.min(list(dict(G.degree).values()))
    d_max = np.max(list(dict(G.degree).values()))
    
    # Check whether mu_2 is calculated with positive version of formula
    if la.norm(np.abs(mu_2)-np.abs((lambda_2*c+np.sqrt((lambda_2*c)**2-4*c*d))/(2*c))) <  1e-8:
        positive.append(G)
        
        # Check whether mu_2 is bounded by replacing x^T(D-I)y by d_bar
        if np.abs(mu_2) <= np.abs((lambda_2*c+np.sqrt((lambda_2*c)**2-4*(d_bar-1)))/(2*c)):
            avg_mu_bound.append(mu_2)
            
        # Check whether mu_2 is bounded by replacing x^T(D-I)y by d_min    
        if np.abs(mu_2) <= np.abs((lambda_2*c+np.sqrt((lambda_2*c)**2-4*(d_min-1)))/(2*c)):
            min_mu_bound.append(mu_2)
        
        # If mu_2 is not bound by replacing x^T(D-I)y with d_min, look at the value of mu_2
        else:
            print(mu_2)
            
    # Check whether mu_2 is calculated with negative version of formula
    elif la.norm(np.abs(mu_2)-np.abs((lambda_2*c-np.sqrt((lambda_2*c)**2-4*c*d))/(2*c))) <  1e-8:
        negative.append(G)
        
        # Check whether mu_2 is bounded by replacing x^T(D-I)y by d_bar
        if np.abs(mu_2) <= np.abs((lambda_2*c-np.sqrt((lambda_2*c)**2-4*(d_bar-1)))/(2*c)):
            avg_mu_bound.append(mu_2)
            
        # Check whether mu_2 is bounded by replacing x^T(D-I)y by d_min
        if np.abs(mu_2) <= np.abs((lambda_2*c-np.sqrt((lambda_2*c)**2-4*(d_min-1)))/(2*c)):
            min_mu_bound.append(mu_2)
            
        # If mu_2 is not bound by replacing x^T(D-I)y with d_min, look at the value of mu_2
        else:
            print(mu_2)
    
    # Collect if mu_2 is not calculated
    else:
        neither.append(G)
    
    # Get eigenvalues and eigenvectors of non-backtracking P
#     rho_vals, rho_vecs = la.eig(nbt.trans_prob_matrix_nb(G))
    
#     # Get second largest of non-backtracking P
#     idx = np.argsort(rho_vals)[-2]
#     rho_2 = rho_vals[idx]
    
    # Check whether mu_2 <= lambda_2/d_min
    if np.abs(mu_2) <= np.abs(lambda_2)/d_min:
        simple_upper.append(G)
        
    # Check whether mu_2 >= lambda_2/d_max
    if np.abs(mu_2) >= np.abs(lambda_2)/d_max:
        simple_lower.append(G)
        
#     # Check whether rho_2 is bound by using the average degree and formula
#     if np.abs(rho_2) <= np.abs((lambda_2*c+np.sqrt((lambda_2*c)**2-4*(d_bar-1)))/(2*c*(d_bar-1))):
#         rho_bound.append(G)
        
    # update the bar
    sys.stdout.write("-")
    sys.stdout.flush()

sys.stdout.write("]\n") # this ends the progress bar

[                                        --------------------------------------------------]


In [5]:
len(positive)

21

In [31]:
len(negative)

29

In [7]:
len(neither)

0

In [8]:
len(avg_mu_bound)

50

In [9]:
len(min_mu_bound)

50

In [10]:
len(simple_lower)

50

In [11]:
len(simple_upper)

0

In [12]:
len(rho_bound)

50

In [13]:
min_mu_bound

[(15.775361899434406+0j),
 (10.679802415834358+0j),
 (5.357142678904788+0j),
 (7.718580995030516+0j),
 (4.6888155409276591+0j),
 (14.348532117621819+0j),
 (4.663803928866689+0j),
 (10.39721229472879+0j),
 (4.355117378173734+0j),
 (11.605704494270277+0j),
 (12.049870455584234+0j),
 (8.0289046552754133+0j),
 (4.892779358200638+0j),
 (5.1889762478755976+0j),
 (5.8313214204315091+0j),
 (15.08280191355203+0j),
 (3.6399744654536992+3.0458912504989417j),
 (6.797635138422403+0j),
 (12.080713083808666+0j),
 (9.4738288624045328+0j),
 (5.6434585292149286+0j),
 (8.7124315365141403+0j),
 (4.5838851220479011+0j),
 (11.802706006650912+0j),
 (3.5753079880676388+0j),
 (10.236721296579308+0j),
 (11.928140594502734+0j),
 (8.7020922061681407+0j),
 (8.8452762045713023+0j),
 (17.043949903886929+0j),
 (6.8645104087173374+0j),
 (16.784540559839492+0j),
 (11.212371113762881+0j),
 (4.1670062153004226+0j),
 (4.6693103365827442+0j),
 (3.633836454935766+3.0767722362216334j),
 (7.4631029214372617+0j),
 (5.481739362

In [19]:
# Check that K is diagonalizable
sy.Matrix(K).is_diagonalizable()

KeyboardInterrupt: 

In [29]:
for i in positive:
    print(f'{len(i.degree)}\t{len(i.edges)}')

172	2215
129	1335
126	1221
88	671
73	373
161	1970
126	1419
149	1645
99	840
98	687
98	750
83	544
81	490
86	542
152	1866
94	681
103	847
108	1108
100	709
133	1465
129	1199


In [28]:
for i in negative:
    print(f'{len(i.degree)}\t{len(i.edges)}')

135	1403
107	978
115	1202
156	1867
119	1317
59	248
127	1167
147	1699
118	1133
128	1455
128	1455
131	1288
119	1238
103	789
97	774
130	1307
116	1003
147	1748
178	2346
99	790
180	2443
134	1280
50	199
116	1276
124	1272
132	1426
123	1130
122	1134
139	1476


In [55]:
ortho = dict()
# Create random graph that is connected
G = nx.random_partition_graph([np.random.randint(20,100),np.random.randint(20,100)],0.25,.05)    # Create adjacency matrix
# Create adjacency matrix, D matrix, D^{-1} matrix
A = nx.adjacency_matrix(G).todense()
D = np.diag(list(dict(G.degree).values()))

# Create K
K = np.block([[A,D-np.eye(D.shape[0])],[-np.eye(D.shape[0]),np.zeros_like(D)]])

# Calculate all y
mu_vals, mu_vecs = la.eig(K)
Y = mu_vecs[mu_vecs.shape[0]//2:,:]

lambda_vals, lambda_vecs = la.eig(A)
for i in range(len(lambda_vals)):
    vecs = []
    for j in range(len(mu_vals)):
#         print(lambda_vecs[:,i].T@Y[:,j])
        if la.norm(lambda_vecs[:,i].T@Y[:,j]) < 1e-5:
            vecs.append(j)
    ortho[i] = vecs

In [ ]:
# G = nx.Graph([[1,2],[2,3],[3,4],[4,5],[2,5],[3,5],[5,6],[6,3]])
G = nx.random_partition_graph([np.random.randint(20,100),np.random.randint(20,100)],0.25,.05)
F = list(nx.connected_component_subgraphs(G))
G = F[0]
# Create adjacency matrix, D matrix, D^{-1} matrix
A = nx.adjacency_matrix(G).todense()
D = np.diag(list(dict(G.degree).values()))

# Create K
K = np.block([[A,D-np.eye(D.shape[0])],[-np.eye(D.shape[0]),np.zeros_like(D)]])

# Calculate all y
mu_vals, mu_vecs = la.eig(K)
Y = mu_vecs[mu_vecs.shape[0]//2:,:]
lambda_vals, lambda_vecs = la.eig(A)

for i in range(len(mu_vals)):
    # Give current mu
    print(f'$\\mu_{i}$: {mu_vals[i]}')
    for k in range(len(lambda_vals)):
        # Check for orthogonality
        if lambda_vecs[:,k].T@Y[:,i] < 1e-8 and lambda_vecs[:,k].T@Y[:,i] > -1e-8:
            print(f"\t$\\lambda_{k}$: Orthogonal")
            continue
        c = lambda_vecs[:,k].T@Y[:,i]
        d = lambda_vecs[:,k].T@(D-np.eye(D.shape[0]))@Y[:,i]
        positive = (lambda_vals[k]*c+np.sqrt((lambda_vals[k]*c)**2-4*c*d))/(2*c)
        negative = (lambda_vals[k]*c-np.sqrt((lambda_vals[k]*c)**2-4*c*d))/(2*c) 
        # Check for positive version
        print(f'\t$\\lambda_{k}$ (positive): {positive}')
        # Check for negative version
        print(f'\t$\\lambda_{k}$ (negative): {negative}')

$\mu_0$: (17.569192882285904+0j)
	$\lambda_0$ (positive): (17.56919288228586+0j)
	$\lambda_0$ (negative): (1.0685946191617715+0j)
	$\lambda_1$ (positive): (17.569192882285165+0j)
	$\lambda_1$ (negative): (-8.24334641562902+0j)
	$\lambda_2$ (positive): (17.569192882286647+0j)
	$\lambda_2$ (negative): (-10.675090330026212+0j)
	$\lambda_3$ (positive): (-24.843290725372118-0j)
	$\lambda_3$ (negative): (17.56919288228681-0j)
	$\lambda_4$ (positive): (-24.68439680961841-0j)
	$\lambda_4$ (negative): (17.569192882288736-0j)
	$\lambda_5$ (positive): (-24.54547503025735-0j)
	$\lambda_5$ (negative): (17.569192882285215-0j)
	$\lambda_6$ (positive): (17.569192882289013+0j)
	$\lambda_6$ (negative): (-24.2017561909843+0j)
	$\lambda_7$ (positive): (17.56919288228662+0j)
	$\lambda_7$ (negative): (-24.044657484072527+0j)
	$\lambda_8$ (positive): (-11.285254727772715-0j)
	$\lambda_8$ (negative): (17.569192882287478-0j)
	$\lambda_9$ (positive): (17.56919288228641+0j)
	$\lambda_9$ (negative): (-11.44980578

	$\lambda_77$ (positive): (5.036723569514605-2.3634939408172495j)
	$\lambda_77$ (negative): (-3.525249107992472+2.3634939408172495j)
	$\lambda_78$ (positive): (2.144103006933741-2.3634939408174076j)
	$\lambda_78$ (negative): (-3.525249107992301+2.3634939408174085j)
	$\lambda_79$ (positive): (-3.525249107992276+2.3634939408174125j)
	$\lambda_79$ (negative): (4.964489828480867-2.363493940817412j)
	$\lambda_80$ (positive): (-3.525249107992382+2.3634939408174556j)
	$\lambda_80$ (negative): (4.835914624314852-2.363493940817455j)
	$\lambda_81$ (positive): (2.2420425140675833-2.363493940817467j)
	$\lambda_81$ (negative): (-3.525249107992315+2.363493940817467j)
	$\lambda_82$ (positive): (2.3287006531983043-2.3634939408173117j)
	$\lambda_82$ (negative): (-3.5252491079924435+2.3634939408173117j)
	$\lambda_83$ (positive): (-3.525249107992327+2.363493940817426j)
	$\lambda_83$ (negative): (4.374237703401715-2.363493940817426j)
	$\lambda_84$ (positive): (4.319571587165248-2.3634939408174245j)
	$\lam

	$\lambda_98$ (negative): (-3.387791106319337+2.228173490830441j)
	$\lambda_99$ (positive): (3.3843986448800347-2.228173490830424j)
	$\lambda_99$ (negative): (-3.38779110631934+2.228173490830424j)
	$\lambda_100$ (positive): (-3.387791106319337+2.2281734908304576j)
	$\lambda_100$ (negative): (3.830524348095451-2.2281734908304576j)
	$\lambda_101$ (positive): (-3.3877911063193054+2.2281734908304416j)
	$\lambda_101$ (negative): (3.4262632633508425-2.2281734908304416j)
	$\lambda_102$ (positive): (-3.3877911063193067+2.2281734908305157j)
	$\lambda_102$ (negative): (2.797649606090028-2.228173490830516j)
	$\lambda_103$ (positive): (2.6986126046539414-2.2281734908304656j)
	$\lambda_103$ (negative): (-3.3877911063194124+2.2281734908304656j)
	$\lambda_104$ (positive): (2.717038789075955-2.2281734908304904j)
	$\lambda_104$ (negative): (-3.387791106319608+2.228173490830491j)
$\mu_9$: (-3.3877911063193746-2.228173490830471j)
	$\lambda_0$ (positive): (-3.387791106323368-2.2281734908305486j)
	$\lambda

	$\lambda_4$ (negative): (-10.112824160126587+2.7722650325688902j)
	$\lambda_5$ (positive): (2.9976202327967805-2.7722650325691998j)
	$\lambda_5$ (negative): (-9.973902380768914+2.7722650325691998j)
	$\lambda_6$ (positive): (2.997620232797206-2.7722650325693787j)
	$\lambda_6$ (negative): (-9.630183541492492+2.772265032569379j)
	$\lambda_7$ (positive): (2.997620232796068-2.772265032569666j)
	$\lambda_7$ (negative): (-9.473084834581977+2.772265032569666j)
	$\lambda_8$ (positive): (3.286317921717659+2.772265032568907j)
	$\lambda_8$ (negative): (2.997620232797104-2.772265032568907j)
	$\lambda_9$ (positive): (3.1217668627124913+2.7722650325689107j)
	$\lambda_9$ (negative): (2.997620232797125-2.7722650325689107j)
	$\lambda_10$ (positive): (-9.179476980660514+2.7722650325689346j)
	$\lambda_10$ (negative): (2.9976202327973205-2.772265032568934j)
	$\lambda_11$ (positive): (3.019782735888563+2.7722650325689298j)
	$\lambda_11$ (negative): (2.9976202327971135-2.7722650325689293j)
	$\lambda_12$ (po

	$\lambda_103$ (negative): (2.3982928780746233-2.1441596745687255j)
	$\lambda_104$ (positive): (-3.087471379740023+2.1441596745686784j)
	$\lambda_104$ (negative): (2.41671906249637-2.144159674568678j)
$\mu_17$: (-3.087471379740103-2.144159674568677j)
	$\lambda_0$ (positive): (21.72525888118754+2.1441596745695177j)
	$\lambda_0$ (negative): (-3.0874713797399025-2.1441596745695177j)
	$\lambda_1$ (positive): (12.413317846396296+2.1441596745687366j)
	$\lambda_1$ (negative): (-3.08747137974015-2.144159674568737j)
	$\lambda_2$ (positive): (9.981573932000536+2.1441596745705453j)
	$\lambda_2$ (negative): (-3.0874713797401054-2.1441596745705462j)
	$\lambda_3$ (positive): (-4.186626463345378+2.1441596745686824j)
	$\lambda_3$ (negative): (-3.087471379739935-2.1441596745686824j)
	$\lambda_4$ (positive): (-4.027732547589573+2.144159674568718j)
	$\lambda_4$ (negative): (-3.0874713797401-2.1441596745687184j)
	$\lambda_5$ (positive): (-3.0874713797400664-2.1441596745686304j)
	$\lambda_5$ (negative): (-

...

	$\lambda_38$ (negative): (-4.412159756251598+3.44432850706634j)
	$\lambda_39$ (positive): (-4.284916538826069+3.444328507066383j)
	$\lambda_39$ (negative): (0.6418430129313132-3.444328507066383j)
	$\lambda_40$ (positive): (0.6418430129314555-3.4443285070661993j)
	$\lambda_40$ (negative): (2.9921902208060636+3.4443285070661993j)
	$\lambda_41$ (positive): (2.905591363600443+3.4443285070662752j)
	$\lambda_41$ (negative): (0.6418430129314213-3.4443285070662752j)
	$\lambda_42$ (positive): (0.6418430129313264-3.444328507066378j)
	$\lambda_42$ (negative): (2.784745105265913+3.4443285070663783j)
	$\lambda_43$ (positive): (-4.161593409346351+3.4443285070664134j)
	$\lambda_43$ (negative): (0.6418430129311979-3.4443285070664134j)
	$\lambda_44$ (positive): (-4.055879328183852+3.444328507066438j)
	$\lambda_44$ (negative): (0.6418430129312991-3.4443285070664387j)
	$\lambda_45$ (positive): (2.6786346301100066+3.4443285070663836j)
	$\lambda_45$ (negative): (0.6418430129312537-3.4443285070663836j)
	$\

	$\lambda_13$ (positive): (0.27869547847827997+3.8804686304470697j)
	$\lambda_13$ (negative): (-6.15109412368757-3.88046863044707j)
	$\lambda_14$ (positive): (-5.983157530330589-3.8804686304474485j)
	$\lambda_14$ (negative): (0.27869547847835624+3.880468630447449j)
	$\lambda_15$ (positive): (0.27869547847818255+3.8804686304475124j)
	$\lambda_15$ (negative): (5.115342516397324-3.8804686304475116j)
	$\lambda_16$ (positive): (0.2786954784782085+3.88046863044767j)
	$\lambda_16$ (negative): (-5.818327911652077-3.8804686304476697j)
	$\lambda_17$ (positive): (0.2786954784782041+3.8804686304473606j)
	$\lambda_17$ (negative): (4.988619883673422-3.880468630447361j)
	$\lambda_18$ (positive): (0.2786954784782224+3.88046863044737j)
	$\lambda_18$ (negative): (4.722116353801018-3.880468630447371j)
	$\lambda_19$ (positive): (-5.705191039815247-3.8804686304474476j)
	$\lambda_19$ (negative): (0.2786954784781739+3.8804686304474476j)
	$\lambda_20$ (positive): (0.27869547847833387+3.8804686304474267j)
	$\l

	$\lambda_44$ (positive): (-3.120835954440494+3.5964076690533147j)
	$\lambda_44$ (negative): (-0.2932003608120596-3.5964076690533147j)
	$\lambda_45$ (positive): (3.6136780038532286+3.5964076690532503j)
	$\lambda_45$ (negative): (-0.2932003608119675-3.59640766905325j)
	$\lambda_46$ (positive): (-2.955534142600477+3.596407669053307j)
	$\lambda_46$ (negative): (-0.2932003608120543-3.596407669053307j)
	$\lambda_47$ (positive): (-0.2932003608120257-3.5964076690532867j)
	$\lambda_47$ (negative): (3.5273191897970237+3.5964076690532867j)
	$\lambda_48$ (positive): (-0.2932003608120132-3.5964076690532787j)
	$\lambda_48$ (negative): (3.452646622077957+3.596407669053279j)
	$\lambda_49$ (positive): (3.3741742888068376+3.596407669053423j)
	$\lambda_49$ (negative): (-0.2932003608119629-3.596407669053423j)
	$\lambda_50$ (positive): (-2.751006752107994+3.596407669053271j)
	$\lambda_50$ (negative): (-0.29320036081200257-3.596407669053271j)
	$\lambda_51$ (positive): (3.1416398469932796+3.5964076690533244

	$\lambda_3$ (positive): (-6.757589398895017-3.662931424564355j)
	$\lambda_3$ (negative): (-0.5165084441902951+3.662931424564356j)
	$\lambda_4$ (positive): (-0.516508444190326+3.6629314245641247j)
	$\lambda_4$ (negative): (-6.5986954831393465-3.6629314245641247j)
	$\lambda_5$ (positive): (-0.5165084441904881+3.6629314245642757j)
	$\lambda_5$ (negative): (-6.4597737037816465-3.6629314245642757j)
	$\lambda_6$ (positive): (-0.5165084441904666+3.6629314245642024j)
	$\lambda_6$ (negative): (-6.116054864504821-3.6629314245642024j)
	$\lambda_7$ (positive): (-5.958956157595435-3.662931424564417j)
	$\lambda_7$ (negative): (-0.5165084441904733+3.6629314245644164j)
	$\lambda_8$ (positive): (6.800446598705307-3.662931424564425j)
	$\lambda_8$ (negative): (-0.5165084441905426+3.6629314245644244j)
	$\lambda_9$ (positive): (-0.5165084441903379+3.662931424564117j)
	$\lambda_9$ (negative): (6.635895539699955-3.662931424564117j)
	$\lambda_10$ (positive): (-0.5165084441903147+3.662931424564273j)
	$\lambda

	$\lambda_93$ (positive): (0.22331025180910624-3.4773187941865773j)
	$\lambda_93$ (negative): (-0.6197979861157115+3.4773187941865773j)
	$\lambda_94$ (positive): (-0.744604559357184+3.4773187941866053j)
	$\lambda_94$ (negative): (0.22331025180915762-3.4773187941866044j)
	$\lambda_95$ (positive): (-0.43283264560498047+3.477318794186603j)
	$\lambda_95$ (negative): (0.2233102518091573-3.477318794186603j)
	$\lambda_96$ (positive): (0.027485900628946767+3.4773187941865857j)
	$\lambda_96$ (negative): (0.22331025180913705-3.4773187941865857j)
	$\lambda_97$ (positive): (-0.3204589901353349+3.477318794186586j)
	$\lambda_97$ (negative): (0.22331025180913908-3.4773187941865866j)
	$\lambda_98$ (positive): (-0.08726231448031585+3.4773187941865875j)
	$\lambda_98$ (negative): (0.22331025180912886-3.4773187941865884j)
	$\lambda_99$ (positive): (-0.22670271324843624+3.477318794186594j)
	$\lambda_99$ (negative): (0.223310251809131-3.4773187941865946j)
	$\lambda_100$ (positive): (0.21942298996697376+3.47

In [3]:
for _ in range(int(100)):
    G = nx.random_partition_graph([5,5],0.25,.05)
    F = list(nx.connected_component_subgraphs(G))
    G = F[0]
    # Create adjacency matrix, D matrix, D^{-1} matrix
    A = nx.adjacency_matrix(G).todense()
    D = np.diag(list(dict(G.degree).values()))

    # Create K
    K = np.block([[A,D-np.eye(D.shape[0])],[-np.eye(D.shape[0]),np.zeros_like(D)]])

    # Calculate all y
    mu_vals, mu_vecs = la.eig(K)
    Y = mu_vecs[mu_vecs.shape[0]//2:,:]
    lambda_vals, lambda_vecs = la.eig(A)
    for i in range(len(mu_vals)):
        # Give current mu
        count = 0
        for k in range(len(lambda_vals)):
            # Check for orthogonality
            if lambda_vecs[:,k].T@Y[:,i] < 1e-8 and lambda_vecs[:,k].T@Y[:,i] > 1e-8:
                print(f'$\mu_{i}$')
        

In [8]:
D

array([[4, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 4]])